In [6]:
import requests
import pandas as pd
import json
import datetime as dt

import os
from dotenv import load_dotenv
load_dotenv("../.env")

False

In [7]:

API_ID = os.environ.get("API_ID")
API_KEY = os.environ.get("API_KEY")
API_SECRET = os.environ.get("API_SECRET")
OPENFX_URL = os.environ.get("OPENFX_URL")

SECURE_HEADER = {
    "Authorization": f"Basic {API_ID}:{API_KEY}:{API_SECRET}",
    "Content-Type": "application/json",
    "Accept": "application/json",
}

In [8]:
session = requests.Session()
session.headers.update(SECURE_HEADER)

In [9]:
full_url = lambda x: f"{OPENFX_URL}/{x}"

In [10]:
full_url('account')

'https://marginalttdemowebapi.fxopen.net/api/v2/account'

In [11]:
resp = session.get(full_url('account'))


In [12]:
resp.status_code

200

In [13]:
print(json.dumps(resp.json(), indent=2))

{
  "Id": 28501564,
  "AccountingType": "Gross",
  "Name": "mauqbK8EqB",
  "FirstName": "",
  "LastName": "",
  "Phone": "+5586981550215",
  "Country": "BRAZIL",
  "State": "",
  "City": "",
  "Address": "",
  "ZipCode": "",
  "Email": "jonatas.iw@gmail.com",
  "Comment": "",
  "Registered": 1725839019376,
  "Modified": 1727472676731,
  "IsArchived": false,
  "IsBlocked": false,
  "IsReadonly": false,
  "IsValid": true,
  "IsWebApiEnabled": true,
  "Leverage": 500,
  "Balance": 9998.96,
  "BalanceCurrency": "USD",
  "Profit": -0.47,
  "Commission": -0.03,
  "AgentCommission": 0.0,
  "Swap": -0.11,
  "Rebate": 0.0,
  "Equity": 9998.36,
  "Margin": 2.24,
  "MarginLevel": 447718.5,
  "MarginCallLevel": 100,
  "StopOutLevel": 50,
  "ReportCurrency": "USD",
  "IsTokenCommissionEnabled": false,
  "Throttling": [
    {
      "Protocol": "Rest",
      "SessionsPerAccount": 2,
      "RequestsPerSecond": 5,
      "ThrottlingMethods": [],
      "ConcurrentRequestCount": 100
    }
  ],
  "IsLongOn

In [14]:
# also we only want symbols where we can also load history data. For that there is the quotehistory/symbols
resp = session.get(full_url('quotehistory/symbols'))
his_symbol_data = resp.json()

print(len(his_symbol_data))
print(his_symbol_data)

# you can probaby see, some of the instruments are appended with "_L"
# in the API code we will filter for symbols that are in the symbol_data and are in his_symbol_data and do not have this L and have StatusGroupId=="Forex"

1300
['#AUS200', '#AUS200_L', '#ESX50', '#ESX50_L', '#FCHI', '#FCHI_L', '#GDAXIm', '#GDAXIm_L', '#HSI', '#HSI_L', '#HSIUSD', '#HSIUSD_L', '#J225', '#J225_L', '#NDXm', '#NDXm_L', '#SPXm', '#SPXm_L', '#UK100', '#UK100_L', '#WS30m', '#WS30m_L', 'A', 'A_L', 'AAL', 'AAL_L', 'AAP', 'AAP_L', 'AAPL', 'AAPL_L', 'AAXN', 'AAXN_L', 'ABBV', 'ABBV_L', 'ABC', 'ABC_L', 'ABT', 'ABT_L', 'ACB', 'ACB_L', 'ACN', 'ACN_L', 'ADBE', 'ADBE_L', 'ADI', 'ADI_L', 'ADM', 'ADM_L', 'ADP', 'ADP_L', 'ADS', 'ADS_L', 'ADSK', 'ADSK_L', 'AEE', 'AEE_L', 'AEP', 'AEP_L', 'AES', 'AES_L', 'AFL', 'AFL_L', 'AGN', 'AGN_L', 'AI', 'AI_L', 'AIG', 'AIG_L', 'AIV', 'AIV_L', 'AIZ', 'AIZ_L', 'AJG', 'AJG_L', 'AKAM', 'AKAM_L', 'ALB', 'ALB_L', 'ALGN', 'ALGN_L', 'ALK', 'ALK_L', 'ALL', 'ALL_L', 'ALLE', 'ALLE_L', 'ALXN', 'ALXN_L', 'AMAT', 'AMAT_L', 'AMCR', 'AMCR_L', 'AMD', 'AMD_L', 'AME', 'AME_L', 'AMG', 'AMG_L', 'AMGN', 'AMGN_L', 'AMP', 'AMP_L', 'AMT', 'AMT_L', 'AMZN', 'AMZN_L', 'ANET', 'ANET_L', 'ANSS', 'ANSS_L', 'AON', 'AON_L', 'AOS', 'AOS_L'

In [16]:
# for a given instrument (symbol) we can get the available candle granularities
resp = session.get(full_url('quotehistory/BTCEUR/periodicities'))
print(resp.json())

['D1', 'H1', 'H4', 'M1', 'M15', 'M30', 'M5', 'MN1', 'S1', 'S10', 'W1']


In [17]:
LABEL_MAP = {
    'Open': 'o',
    'High': 'h',
    'Low': 'l',
    'Close': 'c',
}

# normal params
count = -10
granularity = "M1"
pair = "BTCEUR"

# how far do we need to go back to get our candles
params = dict(
    timestamp=int(pd.Timestamp(dt.datetime.now(dt.UTC)).timestamp() * 1000),
    count=count
)

url = full_url(f'quotehistory/{pair}/{granularity}/bars/bid')
bid_data = session.get(url, params=params).json()

url = full_url(f'quotehistory/{pair}/{granularity}/bars/ask')
ask_data = session.get(url, params=params).json()


In [18]:
bid_data

{'From': 1727482440000,
 'To': 1727482980000,
 'Bars': [{'Timestamp': 1727482440000,
   'Open': 58945.132,
   'High': 58950.636,
   'Low': 58932.481,
   'Close': 58950.621,
   'Volume': 34.0},
  {'Timestamp': 1727482500000,
   'Open': 58950.616,
   'High': 58955.219,
   'Low': 58950.611,
   'Close': 58955.219,
   'Volume': 5.0},
  {'Timestamp': 1727482560000,
   'Open': 58956.224,
   'High': 58961.338,
   'Low': 58955.809,
   'Close': 58956.844,
   'Volume': 29.0},
  {'Timestamp': 1727482620000,
   'Open': 58956.849,
   'High': 58966.485,
   'Low': 58953.305,
   'Close': 58962.281,
   'Volume': 38.0},
  {'Timestamp': 1727482680000,
   'Open': 58962.276,
   'High': 58969.816,
   'Low': 58962.276,
   'Close': 58969.816,
   'Volume': 43.0},
  {'Timestamp': 1727482740000,
   'Open': 58969.919,
   'High': 58969.919,
   'Low': 58958.803,
   'Close': 58963.785,
   'Volume': 29.0},
  {'Timestamp': 1727482800000,
   'Open': 58963.786,
   'High': 58965.786,
   'Low': 58942.458,
   'Close': 58950

In [14]:
# a little utility to take in a candle and return it as and object
# for example, if we are working with bid prices
# price_label='bid'
# item= {'Volume': 1476, 'Close': 1.06064,  'Low': 1.06054,  'High': 1.06104,  'Open': 1.06081,  'Timestamp': 1677535200000}
# the returned object is: { 'time': datetime, 'bid_c': 1.06064,  'bid_l': 1.06054,  'bid_h': 1.06104,  'bid_o': 1.06081 }
def get_price_dict(price_label: str, item):
        data = dict(time=pd.to_datetime(item['Timestamp'], unit='ms'))
        for ohlc in LABEL_MAP.keys():
            data[f"{price_label}_{LABEL_MAP[ohlc]}"]=item[ohlc]
        return data

In [15]:
# let's make the lists of objects
AvailableTo = pd.to_datetime(bid_data['AvailableTo'], unit='ms')

bids = [get_price_dict('bid', item) for item in bid_data["Bars"]]
asks = [get_price_dict('ask', item) for item in ask_data["Bars"]]

In [16]:
# now merge on time - the assumption here is we have the same time values for both. it would be weird if we didn't
df_bid = pd.DataFrame.from_dict(bids)
df_ask = pd.DataFrame.from_dict(asks)
df_merged = pd.merge(left=df_bid, right=df_ask, on='time')   

In [17]:
# FINALLY calcuate the mid, and we are done
for i in ['_o', '_h', '_l', '_c']:
    df_merged[f'mid{i}'] = (df_merged[f'ask{i}'] - df_merged[f'bid{i}']) / 2 + df_merged[f'bid{i}']

In [20]:
if count < 0 and df_merged.shape[0] > 0 and df_merged.iloc[-1].time == AvailableTo:
    df_merged = df_merged[:-1]

In [21]:
df_merged

,time,bid_o,bid_h,bid_l,bid_c,ask_o,ask_h,ask_l,ask_c,mid_o,mid_h,mid_l,mid_c
0,2024-09-09 16:00:00,1.10414,1.10461,1.10414,1.10449,1.10416,1.10460,1.10416,1.10450,1.104150,1.104605,1.104150,1.104495
1,2024-09-09 16:15:00,1.10448,1.10487,1.10445,1.10481,1.10448,1.10489,1.10447,1.10483,1.104480,1.104880,1.104460,1.104820
2,2024-09-09 16:30:00,1.10480,1.10499,1.10470,1.10486,1.10482,1.10501,1.10471,1.10488,1.104810,1.105000,1.104705,1.104870
3,2024-09-09 16:45:00,1.10487,1.10507,1.10474,1.10487,1.10488,1.10509,1.10475,1.10488,1.104875,1.105080,1.104745,1.104875
4,2024-09-09 17:00:00,1.10487,1.10531,1.10481,1.10529,1.10489,1.10527,1.10482,1.10527,1.104880,1.105290,1.104815,1.105280
5,2024-09-09 17:15:00,1.10527,1.10531,1.10512,1.10517,1.10527,1.10527,1.10514,1.10519,1.105270,1.105290,1.105130,1.105180
6,2024-09-09 17:30:00,1.10516,1.10529,1.10489,1.10489,1.10518,1.10527,1.10485,1.10488,1.105170,1.105280,1.104870,1.104885
7,2024-09-09 17:45:00,1.10489,1.10493,1.10434,1.10435,1.10489,1.10494,1.10435,1.10436,1.104890,1.104935,1.104345,1.104355
8,2024-09-09 18:00:00,1.10436,1.10446,1.10414,1.10425,1.10437,1.10448,1.10416,1.10427,1.104365,1.104470,1.104150,1.104260


In [54]:
def fetch_candles(pair, count=-10, granularity="H1"):
    # how far do we need to go back to get our candles
    params = dict(
        timestamp=int(pd.Timestamp(dt.datetime.now(dt.UTC)).timestamp() * 1000),
        count=count
    )
    url = full_url(f'quotehistory/{pair}/{granularity}/bars/bid')
    response_bid_data = session.get(url, params=params)
    bid_data = response_bid_data.json()
    
    if response_bid_data.status_code == 200:
        if 'Bars' not in bid_data:
            bid_data = []
        else:
            bid_data = bid_data


    url = full_url(f'quotehistory/{pair}/{granularity}/bars/ask')
    response_ask_data = session.get(url, params=params)
    ask_data = response_ask_data.json()
    
    if response_ask_data.status_code == 200:
        if 'Bars' not in ask_data:
            ask_data = []
        else:
            ask_data = ask_data

    return response_bid_data.status_code, response_ask_data.status_code, bid_data, ask_data
    
    


In [55]:
def merge_bid_ask(bid_data, ask_data):
    
    bids = [get_price_dict('bid', item) for item in bid_data['Bars']]
    asks = [get_price_dict('ask', item) for item in ask_data['Bars']]
    
    # now merge on time - the assumption here is we have the same time values for both. it would be weird if we didn't
    df_bid = pd.DataFrame.from_dict(bids)
    df_ask = pd.DataFrame.from_dict(asks)
    df_merged = pd.merge(left=df_bid, right=df_ask, on='time')   
    
    # FINALLY calcuate the mid, and we are done
    for i in ['_o', '_h', '_l', '_c']:
        df_merged[f'mid{i}'] = (df_merged[f'ask{i}'] - df_merged[f'bid{i}']) / 2 + df_merged[f'bid{i}']
    
    
    if count < 0 and df_merged.shape[0] > 0 and df_merged.iloc[-1].time == AvailableTo:
        df_merged = df_merged[:-1]

    return df_merged



In [74]:
def create_data_file(pair, count=-10, granularity='H1'):
    code_bid, code_ask, bid_data, ask_data = fetch_candles(pair, count=count, granularity=granularity)
    if (code_bid == 200 and code_ask == 200) and (len(bid_data) == len(ask_data)):
        df_merged = merge_bid_ask(bid_data, ask_data)

    if len(df_merged) == 0:
        print("No candles")
    df_merged.to_pickle(f"./data/{pair}_{granularity}.pkl")
    print(f"{pair} {granularity} {df_merged.shape[0]} {df_merged.time.min()} {df_merged.time.max()}")

In [75]:
# code_bid, code_ask, bid_data, ask_data = fetch_candles('EURUSD', count=-20, granularity='M15')

In [76]:
# if (code_bid == 200 and code_ask == 200) and (len(bid_data) == len(ask_data)):
#     df_merged = merge_bid_ask(bid_data, ask_data)

In [81]:
# create_data_file("EURUSD", count=-10, granularity='H1')

In [82]:
our_cur = ['USD', 'GBP', 'JPY', 'EUR']
for p1 in our_cur:
    for p2 in our_cur:
        pr = f"{p1}{p2}"
        if pr in his_symbol_data:
            create_data_file(pr, count=-1000, granularity='H1')

USDJPY H1 1000 2024-07-12 04:00:00 2024-09-09 19:00:00
GBPUSD H1 1000 2024-07-12 04:00:00 2024-09-09 19:00:00
GBPJPY H1 1000 2024-07-12 04:00:00 2024-09-09 19:00:00
EURUSD H1 1000 2024-07-12 04:00:00 2024-09-09 19:00:00
EURGBP H1 1000 2024-07-12 04:00:00 2024-09-09 19:00:00
EURJPY H1 1000 2024-07-12 04:00:00 2024-09-09 19:00:00
